# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import scipy.sparse as sps
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Time-delay model

$$
\begin{align*}
  E \dot{x}(t) & = A_0 x(t) + A_1 x(t - \tau) + B u(t) \\
  y(t) & = C x(t)
\end{align*}
$$

In [ ]:
from pymor.models.iosys import LinearDelayModel
from pymor.operators.numpy import NumpyMatrixOperator

In [ ]:
mat = spio.loadmat('data/DelayData.mat')
mat.keys()

In [ ]:
E = mat['E']
A0 = mat['A0']
A1 = mat['A1']
B = mat['b'].toarray()
C = mat['c'].toarray()
tau = float(mat['tau'][0, 0])

Eop = NumpyMatrixOperator(E)
A0op = NumpyMatrixOperator(A0)
A1op = NumpyMatrixOperator(A1)
Bop = NumpyMatrixOperator(B)
Cop = NumpyMatrixOperator(C)

In [ ]:
fom = LinearDelayModel(A0op, (A1op,), (tau,), Bop, Cop, E=Eop)

# Bode plot

Plot the Bode and/or magnitude plot.

# Transfer function IRKA (TF-IRKA)

TF-IRKA is an extension of IRKA where the full-order model can be any real LTI system represented by a transfer function. For interpolation, it uses Loewner matrices, compared to IRKA which uses (tangential) rational Krylov subspaces.

In pyMOR, TF-IRKA is implemented in `TFIRKAReductor`, which can take any model with `eval_tf` and `eval_dtf` methods. Currently, those are `LTIModel`, `SecondOrderModel`, `TransferFunction`, and, the one relevant here, `LinearDelayModel`.

The resulting reduced-order model will be an `LTIModel`, i.e., it will not preserve the structure of the full-order model.

Run TF-IRKA for the `fom` model defined above and draw the Bode/magnitude plots of the full-order model and the reduced-order model.

In [ ]:
from pymor.reductors.h2 import TFIRKAReductor

# Structure-preserving interpolation

The obtain a reduced-order model which is again a `LinearDelayModel`, we can use structure-preserving bitangential Hermite interpolation implemented in `DelayBHIReductor` from `pymor.reductors.interpolation`. Interpolate the full-order model at a few points on the imaginary axis and plot the results.

In [ ]:
from pymor.reductors.interpolation import DelayBHIReductor